In [218]:
from typing import List, Optional, Dict, Union

import pandas as pd
import json
import numpy as np
from dataclasses import dataclass

pd.set_option('max_colwidth', None)

In [34]:
df_ref = pd.read_excel("./data/karti competencii_marketing.xlsx", skiprows = [0])

Достаем мэтчинг характеристик.

In [100]:
def clean_start_number(text: str):
    if type(text) != str:
        return text
    while not text[0].isalpha():
        text = text[1:]
    return text

In [101]:
with open("./save/competencies.json", "r") as f:
    dict_maincomp_comp = json.load(f)

all_competencies = []
for key in dict_maincomp_comp:
    if len(dict_maincomp_comp[key]) == 0:
        dict_maincomp_comp[key].append(key)
    dict_maincomp_comp[key] = [clean_start_number(comp) for comp in dict_maincomp_comp[key]]
    all_competencies += dict_maincomp_comp[key]

In [102]:
queries_mapping = {
    "знания": "Unnamed: 8",
    "должности": "Unnamed: 5",
    "навыки": "Unnamed: 11"
}

In [163]:
def compose_match_dict(
    df: pd.DataFrame,
    col_parent: str,
    col_child: str,
    filter_parent_values: Optional[List[str]] = None
):
    df_copy = df.copy()
    if filter_parent_values is not None:
        df_copy = df_copy[df_copy[col_parent].apply(clean_start_number).isin(filter_parent_values)]
    inds_nna = df_copy[~df_copy[col_parent].isna()].index.values
    res_dict = {}
    for i in range(1, len(inds_nna)):
        key = df.loc[inds_nna[i - 1]][col_parent]
        values = df.loc[inds_nna[i - 1] : inds_nna[i] - 1][col_child].dropna().values
        if type(key) == str:
            values = [val.strip() for val in values if type(val) == str]
            if key not in res_dict:
                res_dict[clean_start_number(key).strip()] = values
            else:
                res_dict[clean_start_number(key).strip()] += values
                res_dict[clean_start_number(key).strip()] = list(np.unique(res_dict[clean_start_number(key).strip()]))
    return res_dict

In [165]:
dict_know_skills = compose_match_dict(
    df=df_ref,
    col_parent=queries_mapping["знания"],
    col_child=queries_mapping["навыки"]
)
dict_comp_know = compose_match_dict(
    df=df_ref,
    col_parent="Трудовые функции / компетенции",
    col_child=queries_mapping["знания"],
    filter_parent_values=all_competencies
)

In [166]:
@dataclass
class Knowledge:
    name: str
    skills: List[str]

@dataclass
class Competence:
    name: str
    knowledge: List[Knowledge]

@dataclass
class MainCompetence:
    name: str
    competencies: List[Competence]

# Маркетолог

In [325]:
df_comp = pd.read_csv("./save/comp_matching_compare.csv")
df_skills = pd.read_csv("./save/match_results_навыки_ext.csv")
df_know = pd.read_csv("./save/match_results_знания_ext.csv")
df_position = pd.read_csv("./save/match_results_должности.csv")
skills = [
    'Системное мышление',
    'Стратегическое мышление',
    'Добросовестность',
    'Соблюдение трудовой дисциплины',
    'Организационные способности',
    'Способность к анализу и контролю',
    'Умение разрабатывать и реализовывать стратегии',
    'Навыки взаимодействия с сотрудниками и руководством',
    'Способность работать в команде',
    'Навыки ведения деловых коммуникаций',
    'Умение проводить эффективные мониторинги и анализы'
]

In [350]:
perc_check = np.percentile(df_skills["cosine_sim"].values, 7)

In [351]:
df_skills[df_skills['cosine_sim'] < perc_check]

,Найденные навыки,Существующие навыки,cosine_sim,Соответствие
30,Грамотная речь,Владение русским языком,0.513,Хорошо
31,Высокий уровень самоорганизации и ответственности,Разрабатывать и принимать управленческие решения,0.473,Норм
33,Системное мышление,Анализ среды для коммуникации и навыки работы с постоянно меняющейся средой,0.445,Норм


In [352]:
new_skills = df_skills[df_skills['cosine_sim'] < perc_check]["Найденные навыки"].values

In [353]:
skills_found = df_skills[df_skills['cosine_sim'] > perc_check]["Существующие навыки"].values

In [354]:
perc_check = np.percentile(df_know["cosine_sim"].values, 7)

In [355]:
df_know[df_know['cosine_sim'] < perc_check]

,Найденные знания,Существующие знания,cosine_sim,Соответствие
17,принципы системного анализа,Принципы и методы бизнес-планирования,0.475,Не совпадает
18,психологические особенности поведения людей разных возрастов в различных жизненных ситуациях,Основные поведенческие факторы,0.443,Идеально


In [356]:
know_found = df_know[df_know['cosine_sim'] > perc_check]["Существующие знания"].values

In [357]:
new_know = df_know[df_know['cosine_sim'] < perc_check]["Найденные знания"].values

In [358]:
perc_check = np.percentile(df_comp["cosine_sim cosine"].values, 7)

In [359]:
df_comp[df_comp['cosine_sim cosine'] < perc_check][["Найденные компетенции", "Существующие компетенции cosine", "cosine_sim cosine", "Существующие компетенции combined", "Соответствие"]]

,Найденные компетенции,Существующие компетенции cosine,cosine_sim cosine,Существующие компетенции combined,Соответствие
9,Организация участия компании в профильных выставках,Реализация маркетинговых и рекламных мероприятий,0.493,Реализация маркетинговых и рекламных мероприятий,Норм
19,Работа с подрядчиками,Работа со стажерами,0.530,Работа со стажерами,Не соответствует


In [360]:
comp_found = df_comp[df_comp['cosine_sim cosine'] > perc_check]["Существующие компетенции cosine"].apply(lambda x: x.strip()).values

In [362]:
new_comps = df_comp[df_comp['cosine_sim cosine'] < perc_check]["Найденные компетенции"].values

In [286]:
def get_found_dict(dict_match : Dict, values_found: List[Union[str, Knowledge, Competence]]):
    res_dict = {}
    for val in values_found:
        found = False
        val_str = val
        if type(val) != str:
            val_str = val.name
        val_str = val_str.strip()
        for key in dict_match:
            if val_str in dict_match[key]:
                if key not in res_dict:
                    res_dict[key] = []
                found = True
                if val not in res_dict[key]:
                    res_dict[key].append(val)
                break
        if found == False:
            print(val_str)
    return res_dict

In [253]:
know_dict = get_found_dict(dict_know_skills, skills_found)

In [254]:
set(know_found).intersection(know_dict.keys())

{'Методы разработки товарной политики'}

In [255]:
know_found_not_matched_str = list(set(know_found) - set(know_found).intersection(know_dict.keys()))
know_found_not_matched = [Knowledge(name=know, skills=[]) for know in know_found_not_matched_str]

In [256]:
know_list_matched = []
know_list_not_matched = []
for know in know_dict:
    if know in know_found:
        know_list_matched.append(Knowledge(name=know, skills=know_dict[know]))
    else:
        know_list_not_matched.append(Knowledge(name=know, skills=know_dict[know]))

In [257]:
print("Найдено, но нет в скиллах: ", len(know_found_not_matched))
print("Найдено, есть в скиллах: ", len(know_list_matched))
print("Не найдено, есть в скиллах: ", len(know_list_not_matched))

Найдено, но нет в скиллах:  15
Найдено, есть в скиллах:  1
Не найдено, есть в скиллах:  17


In [258]:
full_knowledge = know_found_not_matched + know_list_matched + know_list_not_matched

In [279]:
comp_dict = get_found_dict(dict_comp_know, full_knowledge)

In [280]:
set(comp_found).intersection(comp_dict.keys())

{'Поиск, сбор и анализ маркетинговой информации',
 'Проводить маркетинговые исследования и анализировать результаты',
 'Разработка ценовой политики компании',
 'Реализация маркетинговых и рекламных мероприятий'}

In [281]:
comp_found_not_matched_str = list(set(comp_found) - set(comp_found).intersection(comp_dict.keys()))
comp_found_not_matched = [Competence(name=comp, knowledge=[]) for comp in comp_found_not_matched_str]

In [282]:
comp_list_matched = []
comp_list_not_matched = []
for comp in comp_dict:
    if comp in comp_found:
        comp_list_matched.append(Competence(name=comp, knowledge=comp_dict[comp]))
    else:
        comp_list_not_matched.append(Competence(name=comp, knowledge=comp_dict[comp]))

In [283]:
print("Найдено, но нет в навыках: ", len(comp_found_not_matched))
print("Найдено, есть в навыках: ", len(comp_list_matched))
print("Не найдено, есть в навыках: ", len(comp_list_not_matched))

Найдено, но нет в навыках:  11
Найдено, есть в навыках:  4
Не найдено, есть в навыках:  20


In [284]:
full_competencies = comp_found_not_matched + comp_list_matched + comp_list_not_matched

In [287]:
main_comp_dict = get_found_dict(dict_maincomp_comp, full_competencies)

In [289]:
main_comp_list = [MainCompetence(name=key, competencies=main_comp_dict[key]) for key in main_comp_dict]

In [296]:
def create_comp_df(main_comp: MainCompetence):
    rows = []
    for comp in main_comp.competencies:
        if len(comp.knowledge) == 0:
            row = [main_comp.name, comp.name, "", ""]
            rows.append(row)
        for know in comp.knowledge:
            if len(know.skills) == 0:
                row = [main_comp.name, comp.name, know.name, ""]
                rows.append(row)
            for skill in know.skills:
                row = [main_comp.name, comp.name, know.name, skill]
                rows.append(row)
    df = pd.DataFrame(np.vstack(rows), columns=["Основная компетенция", "Компетенция", "Знания", "Навыки"])
    return df

In [303]:
all_dfs = []
for maincomp in main_comp_list:
    df = create_comp_df(maincomp)
    all_dfs.append(df)

In [307]:
df_all = pd.concat(all_dfs).reset_index(drop=True)

In [319]:
def clean_column(values: np.array):
    prev_value = values[0]
    for i in range(1, len(values)):
        if values[i] == prev_value:
            values[i] = ""
        else:
            prev_value = values[i]
    return values

In [321]:
for col in df_all.columns[:-1]:
    df_all[col] = clean_column(df_all[col].values)

In [335]:
position = df_position.iloc[0]["Существующие должности"]
position_level = df_ref[df_ref[queries_mapping['должности']] == position]["Уровень должности"].iloc[0]

In [336]:
df_all["Софт-скиллы"] = skills + [""] * (df_all.shape[0] - len(skills))
df_all["Направление (сфера)"] = ["Маркетинг"] + [""] * (df_all.shape[0] - 1)
df_all["Должность"] = [position] + [""] * (df_all.shape[0] - 1)
df_all["Уровень должности"] = [position_level] + [""] * (df_all.shape[0] - 1)

In [343]:
df_all = df_all[
    list(df_all.columns[-3:]) + list(df_all.columns[:-3]) 
]

In [345]:
df_all.to_csv("./save/knk_marketolog.csv", index = False)

In [363]:
df_all.head()

,Направление (сфера),Должность,Уровень должности,Основная компетенция,Компетенция,Знания,Навыки,Софт-скиллы
0,Маркетинг,Маркетолог,UD02,Вывод на рынок нового продукта,"Разработка нового продукта: идея, прототип, уточнение требований, тестирование и управление качеством",,,Системное мышление
1,,,,,Реализация маркетинговых и рекламных мероприятий по продвижению нового продукта,,,Стратегическое мышление
2,,,,,Оценка привлекательности нового продукта,Методика оценки привлекательности новой продукции для компании; методика оценки рисков внешней среды,,Добросовестность
3,,,,,Планирование продаж. Технология разработки сбытовой сети для нового продукта,Формирование системы продаж и стратегии сбыта,,Соблюдение трудовой дисциплины
4,,,,,Маркетинговый анализ на прединвестиционном этапе. Анализ рынка и маркетинговой среды при разработке нового продукта,Анализ рынка: техническое задание на заказ исследования,Создание технического задания на заказ исследования рынка,Организационные способности


In [365]:
max_len_new = max(list(map(len, [new_comps, new_know, new_skills])))

In [366]:
cols = []
for values in [new_comps, new_know, new_skills]:
    cols.append(list(values) + [""] * (max_len_new - len(values)))

In [369]:
new_char = pd.DataFrame(np.vstack(cols).T, columns = ["Компетенции", "Знания", "Навыки"])

In [371]:
new_char.to_csv("./save/new_characteristics_candidates.csv", index = False)